In [22]:
import psycopg2

try:
    conn = psycopg2.connect("dbname='db_alquran' user='postgres' host='localhost' password='12345'")
except:
    print("I am unable to connect to the database")

In [23]:
import psycopg2.extras

cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)

In [24]:
cur.execute("""select id, chapter_num, verse_num, lem from alquran_trans_full order by chapter_num, id, verse_num""")

In [25]:
rows = cur.fetchall()

In [26]:
rows

[[6237, 1, 1, 'bi {som {ll~ah {l r~aHoma`n {l r~aHiym'],
 [6238, 1, 2, '{lo Hamod li {ll~ah rab~ {lo Ea`lamiyn'],
 [6239, 1, 3, '{l r~aHoma`n {l r~aHiym'],
 [6240, 1, 4, 'ma`lik yawom {l diyn'],
 [6241, 1, 5, '<iy~aA Eabada wa <iy~aA {sotaEiynu'],
 [6242, 1, 6, 'hadaY naA {l Sira`T {lo m~usotaqiym'],
 [6243,
  1,
  7,
  'Sira`T {l~a*iY >anoEama ta EalaY` himo gayor {lo magoDuwb EalaY` himo wa laA {l DaA^l~'],
 [6244, 2, 1, 'Al^m^'],
 [6245, 2, 2, '*a`lik {lo kita`b laA rayob fiY hi hudFY l~i lo mut~aqiyn'],
 [6246,
  2,
  3,
  "{l~a*iY 'aAmana wna bi {lo gayob wa >aqaAma wna {l Salaw`p wa min maA razaqa na` humo >anfaqa wna"],
 [6247,
  2,
  4,
  "wa {l~a*iY 'aAmana wna bi maA >anzala <ilaY` ka wa maA >anzala min qabol ka wa bi {lo A^xir humo yuwqinu wna"],
 [6248,
  2,
  5,
  '>uwla`^}ik EalaY` hudFY min rab~ himo wa >uwla`^}ik humu {lo mufoliHuwn'],
 [6249,
  2,
  6,
  "<in~ {l~a*iY kafara wA@ sawaA^' EalaY` himo 'a >an*ara ta humo >am lam >an*ara humo laA 'aAmana wna"],
 [6250,
  2,

In [27]:
rows_process = []
print("\nShow me the database:\n")
for row in rows:
    terjemah = row['lem']
#     terjemah_low = terjemah.lower()
    terjemah_token = terjemah.split(' ')
    terjemah_tanda = ['['+ x + ']' for x in terjemah_token]
    terjemah_array = ' '.join(terjemah_tanda)
#     print(" ", row['id'])
#     print(" ", row['surat'])
#     print(" ", row['ayat'])
#     print(" ", terjemah_ganti_kurung)
    row_new = {
        'id': row['id'],
        'chapter_num': row['chapter_num'],
        'verse_num': row['verse_num'],
        'form': terjemah,
        'terjemah_preprocess': terjemah_array
    }
    rows_process.append(row_new)
rows_process


Show me the database:



[{'id': 6237,
  'chapter_num': 1,
  'verse_num': 1,
  'form': 'bi {som {ll~ah {l r~aHoma`n {l r~aHiym',
  'terjemah_preprocess': '[bi] [{som] [{ll~ah] [{l] [r~aHoma`n] [{l] [r~aHiym]'},
 {'id': 6238,
  'chapter_num': 1,
  'verse_num': 2,
  'form': '{lo Hamod li {ll~ah rab~ {lo Ea`lamiyn',
  'terjemah_preprocess': '[{lo] [Hamod] [li] [{ll~ah] [rab~] [{lo] [Ea`lamiyn]'},
 {'id': 6239,
  'chapter_num': 1,
  'verse_num': 3,
  'form': '{l r~aHoma`n {l r~aHiym',
  'terjemah_preprocess': '[{l] [r~aHoma`n] [{l] [r~aHiym]'},
 {'id': 6240,
  'chapter_num': 1,
  'verse_num': 4,
  'form': 'ma`lik yawom {l diyn',
  'terjemah_preprocess': '[ma`lik] [yawom] [{l] [diyn]'},
 {'id': 6241,
  'chapter_num': 1,
  'verse_num': 5,
  'form': '<iy~aA Eabada wa <iy~aA {sotaEiynu',
  'terjemah_preprocess': '[<iy~aA] [Eabada] [wa] [<iy~aA] [{sotaEiynu]'},
 {'id': 6242,
  'chapter_num': 1,
  'verse_num': 6,
  'form': 'hadaY naA {l Sira`T {lo m~usotaqiym',
  'terjemah_preprocess': '[hadaY] [naA] [{l] [Sira`T] [{lo]

In [28]:
from suffix_trees import STree

list_hasil = []

print("\nShow me the database:\n")
for idx, row in enumerate(rows_process[0:300]):
    for idx, row2 in enumerate(rows_process[0:300]):
        if row['id'] < row2['id']:
            a = [row['terjemah_preprocess'], row2['terjemah_preprocess']]
            st = STree.STree(a)
            lcs = st.lcs()
            lcs_split = lcs.split(' ')
            filter_lcs = list(filter(lambda kata: kata.startswith("[") and kata.endswith("]"), lcs_split))
#             b = list(filter(lambda kata: len(kata) >= 2, lcs.split(' ')))
            jumlah_kata = len(filter_lcs)
#             print(jumlah_kata)
            if jumlah_kata >= 3:
#                 print(" ", idx, row['id'], " ", row2['id'])
#                 print(" ", idx, row['surat'], " ", row2['surat'])
#                 print(" ", idx, row['ayat'], " ", row2['ayat'])
#                 print(" ", idx, row['terjemah'], " ", row2['terjemah'])
#                 print(b)
                hilangkan_marker = [x[1:len(x)-1] for x in filter_lcs]
                terjemah_string = ' '.join(hilangkan_marker)
                print("Surat: ", row['chapter_num'],"Surat Pembanding: ", row2['chapter_num'], "Ayat: ", row['verse_num'], "Ayat Pembanding: ", row2['verse_num'], "LCS: ", terjemah_string)
                hasil = [row['chapter_num'], row2['chapter_num'], row['verse_num'], row2['verse_num'], jumlah_kata, terjemah_string]
                list_hasil.append(hasil)
        
print(len(list_hasil))


Show me the database:

Surat:  1 Surat Pembanding:  1 Ayat:  1 Ayat Pembanding:  3 LCS:  {l r~aHoma`n {l r~aHiym
Surat:  1 Surat Pembanding:  2 Ayat:  1 Ayat Pembanding:  163 LCS:  {l r~aHoma`n {l r~aHiym
Surat:  1 Surat Pembanding:  2 Ayat:  2 Ayat Pembanding:  131 LCS:  rab~ {lo Ea`lamiyn
Surat:  1 Surat Pembanding:  2 Ayat:  3 Ayat Pembanding:  163 LCS:  {l r~aHoma`n {l r~aHiym
Surat:  1 Surat Pembanding:  2 Ayat:  7 Ayat Pembanding:  38 LCS:  EalaY` himo wa laA
Surat:  1 Surat Pembanding:  2 Ayat:  7 Ayat Pembanding:  62 LCS:  EalaY` himo wa laA
Surat:  1 Surat Pembanding:  2 Ayat:  7 Ayat Pembanding:  112 LCS:  EalaY` himo wa laA
Surat:  1 Surat Pembanding:  2 Ayat:  7 Ayat Pembanding:  120 LCS:  wa laA {l
Surat:  1 Surat Pembanding:  2 Ayat:  7 Ayat Pembanding:  160 LCS:  EalaY` himo wa
Surat:  1 Surat Pembanding:  2 Ayat:  7 Ayat Pembanding:  167 LCS:  EalaY` himo wa
Surat:  1 Surat Pembanding:  2 Ayat:  7 Ayat Pembanding:  262 LCS:  EalaY` himo wa laA
Surat:  1 Surat Pembandin

Surat:  2 Surat Pembanding:  2 Ayat:  9 Ayat Pembanding:  62 LCS:  {l~a*iY 'aAmana wA@ wa
Surat:  2 Surat Pembanding:  2 Ayat:  9 Ayat Pembanding:  76 LCS:  {l~a*iY 'aAmana wA@
Surat:  2 Surat Pembanding:  2 Ayat:  9 Ayat Pembanding:  82 LCS:  wa {l~a*iY 'aAmana wA@ wa
Surat:  2 Surat Pembanding:  2 Ayat:  9 Ayat Pembanding:  103 LCS:  'aAmana wA@ wa
Surat:  2 Surat Pembanding:  2 Ayat:  9 Ayat Pembanding:  104 LCS:  {l~a*iY 'aAmana wA@
Surat:  2 Surat Pembanding:  2 Ayat:  9 Ayat Pembanding:  145 LCS:  humo wa maA
Surat:  2 Surat Pembanding:  2 Ayat:  9 Ayat Pembanding:  153 LCS:  {l~a*iY 'aAmana wA@
Surat:  2 Surat Pembanding:  2 Ayat:  9 Ayat Pembanding:  165 LCS:  {ll~ah wa {l~a*iY 'aAmana
Surat:  2 Surat Pembanding:  2 Ayat:  9 Ayat Pembanding:  172 LCS:  {l~a*iY 'aAmana wA@
Surat:  2 Surat Pembanding:  2 Ayat:  9 Ayat Pembanding:  178 LCS:  {l~a*iY 'aAmana wA@
Surat:  2 Surat Pembanding:  2 Ayat:  9 Ayat Pembanding:  183 LCS:  {l~a*iY 'aAmana wA@
Surat:  2 Surat Pembanding:  2 Ay

Surat:  2 Surat Pembanding:  2 Ayat:  14 Ayat Pembanding:  218 LCS:  {l~a*iY 'aAmana wA@
Surat:  2 Surat Pembanding:  2 Ayat:  14 Ayat Pembanding:  249 LCS:  {l~a*iY 'aAmana wA@
Surat:  2 Surat Pembanding:  2 Ayat:  14 Ayat Pembanding:  257 LCS:  {l~a*iY 'aAmana wA@
Surat:  2 Surat Pembanding:  2 Ayat:  14 Ayat Pembanding:  264 LCS:  {l~a*iY 'aAmana wA@
Surat:  2 Surat Pembanding:  2 Ayat:  14 Ayat Pembanding:  275 LCS:  qaAla w^A@ <in~
Surat:  2 Surat Pembanding:  2 Ayat:  14 Ayat Pembanding:  277 LCS:  {l~a*iY 'aAmana wA@
Surat:  2 Surat Pembanding:  2 Ayat:  14 Ayat Pembanding:  278 LCS:  {l~a*iY 'aAmana wA@
Surat:  2 Surat Pembanding:  2 Ayat:  16 Ayat Pembanding:  33 LCS:  wa maA kaAna
Surat:  2 Surat Pembanding:  2 Ayat:  16 Ayat Pembanding:  59 LCS:  maA kaAna wA@
Surat:  2 Surat Pembanding:  2 Ayat:  16 Ayat Pembanding:  86 LCS:  >uwla`^}ik {l~a*iY {$otaraY` wuA@
Surat:  2 Surat Pembanding:  2 Ayat:  16 Ayat Pembanding:  113 LCS:  maA kaAna wA@
Surat:  2 Surat Pembanding:  2 Ay

Surat:  2 Surat Pembanding:  2 Ayat:  25 Ayat Pembanding:  114 LCS:  wa la humo fiY
Surat:  2 Surat Pembanding:  2 Ayat:  25 Ayat Pembanding:  153 LCS:  {l~a*iY 'aAmana wA@
Surat:  2 Surat Pembanding:  2 Ayat:  25 Ayat Pembanding:  172 LCS:  {l~a*iY 'aAmana wA@
Surat:  2 Surat Pembanding:  2 Ayat:  25 Ayat Pembanding:  174 LCS:  wa la humo
Surat:  2 Surat Pembanding:  2 Ayat:  25 Ayat Pembanding:  178 LCS:  {l~a*iY 'aAmana wA@
Surat:  2 Surat Pembanding:  2 Ayat:  25 Ayat Pembanding:  183 LCS:  {l~a*iY 'aAmana wA@
Surat:  2 Surat Pembanding:  2 Ayat:  25 Ayat Pembanding:  189 LCS:  wa >ataY wA@
Surat:  2 Surat Pembanding:  2 Ayat:  25 Ayat Pembanding:  208 LCS:  {l~a*iY 'aAmana wA@
Surat:  2 Surat Pembanding:  2 Ayat:  25 Ayat Pembanding:  212 LCS:  {l~a*iY 'aAmana wA@ wa
Surat:  2 Surat Pembanding:  2 Ayat:  25 Ayat Pembanding:  213 LCS:  {l~a*iY 'aAmana wA@
Surat:  2 Surat Pembanding:  2 Ayat:  25 Ayat Pembanding:  214 LCS:  {l~a*iY 'aAmana wA@
Surat:  2 Surat Pembanding:  2 Ayat:  2

Surat:  2 Surat Pembanding:  2 Ayat:  29 Ayat Pembanding:  231 LCS:  bi kul~ $aYo' Ealiym
Surat:  2 Surat Pembanding:  2 Ayat:  29 Ayat Pembanding:  255 LCS:  maA fiY {lo >aroD
Surat:  2 Surat Pembanding:  2 Ayat:  29 Ayat Pembanding:  273 LCS:  fiY {lo >aroD
Surat:  2 Surat Pembanding:  2 Ayat:  29 Ayat Pembanding:  282 LCS:  bi kul~ $aYo' Ealiym
Surat:  2 Surat Pembanding:  2 Ayat:  29 Ayat Pembanding:  284 LCS:  maA fiY {lo >aroD
Surat:  2 Surat Pembanding:  3 Ayat:  29 Ayat Pembanding:  5 LCS:  fiY {lo >aroD
Surat:  2 Surat Pembanding:  2 Ayat:  30 Ayat Pembanding:  33 LCS:  <in~ Y^ Ealima
Surat:  2 Surat Pembanding:  2 Ayat:  30 Ayat Pembanding:  34 LCS:  li lo malak
Surat:  2 Surat Pembanding:  2 Ayat:  30 Ayat Pembanding:  36 LCS:  fiY {lo >aroD
Surat:  2 Surat Pembanding:  2 Ayat:  30 Ayat Pembanding:  54 LCS:  wa <i* qaAla
Surat:  2 Surat Pembanding:  2 Ayat:  30 Ayat Pembanding:  55 LCS:  wa <i* qaAla
Surat:  2 Surat Pembanding:  2 Ayat:  30 Ayat Pembanding:  58 LCS:  wa <i* 

...

Surat:  2 Surat Pembanding:  2 Ayat:  209 Ayat Pembanding:  211 LCS:  min baEod maA jaA^'a
Surat:  2 Surat Pembanding:  2 Ayat:  209 Ayat Pembanding:  213 LCS:  min baEod maA jaA^'a
Surat:  2 Surat Pembanding:  2 Ayat:  209 Ayat Pembanding:  220 LCS:  {ll~ah Eaziyz Hakiym
Surat:  2 Surat Pembanding:  2 Ayat:  209 Ayat Pembanding:  223 LCS:  Ealima w^A@ >an~
Surat:  2 Surat Pembanding:  2 Ayat:  209 Ayat Pembanding:  228 LCS:  {ll~ah Eaziyz Hakiym
Surat:  2 Surat Pembanding:  2 Ayat:  209 Ayat Pembanding:  231 LCS:  Ealima w^A@ >an~ {ll~ah
Surat:  2 Surat Pembanding:  2 Ayat:  209 Ayat Pembanding:  233 LCS:  Ealima w^A@ >an~ {ll~ah
Surat:  2 Surat Pembanding:  2 Ayat:  209 Ayat Pembanding:  235 LCS:  Ealima w^A@ >an~ {ll~ah
Surat:  2 Surat Pembanding:  2 Ayat:  209 Ayat Pembanding:  240 LCS:  {ll~ah Eaziyz Hakiym
Surat:  2 Surat Pembanding:  2 Ayat:  209 Ayat Pembanding:  244 LCS:  Ealima w^A@ >an~ {ll~ah
Surat:  2 Surat Pembanding:  2 Ayat:  209 Ayat Pembanding:  253 LCS:  min baEod ma

Surat:  2 Surat Pembanding:  2 Ayat:  218 Ayat Pembanding:  257 LCS:  {l~a*iY 'aAmana wA@
Surat:  2 Surat Pembanding:  2 Ayat:  218 Ayat Pembanding:  261 LCS:  fiY sabiyl {ll~ah
Surat:  2 Surat Pembanding:  2 Ayat:  218 Ayat Pembanding:  262 LCS:  fiY sabiyl {ll~ah
Surat:  2 Surat Pembanding:  2 Ayat:  218 Ayat Pembanding:  264 LCS:  {l~a*iY 'aAmana wA@
Surat:  2 Surat Pembanding:  2 Ayat:  218 Ayat Pembanding:  273 LCS:  wA@ fiY sabiyl {ll~ah
Surat:  2 Surat Pembanding:  2 Ayat:  218 Ayat Pembanding:  277 LCS:  <in~ {l~a*iY 'aAmana wA@ wa
Surat:  2 Surat Pembanding:  2 Ayat:  218 Ayat Pembanding:  278 LCS:  {l~a*iY 'aAmana wA@
Surat:  2 Surat Pembanding:  2 Ayat:  218 Ayat Pembanding:  282 LCS:  {ll~ah wa {ll~ah
Surat:  2 Surat Pembanding:  2 Ayat:  219 Ayat Pembanding:  220 LCS:  sa>ala wna ka Ean {lo
Surat:  2 Surat Pembanding:  2 Ayat:  219 Ayat Pembanding:  221 LCS:  li l n~aAs
Surat:  2 Surat Pembanding:  2 Ayat:  219 Ayat Pembanding:  222 LCS:  sa>ala wna ka Ean {lo
Surat:  2 Su

Surat:  2 Surat Pembanding:  2 Ayat:  231 Ayat Pembanding:  251 LCS:  wa {lo Hikomap
Surat:  2 Surat Pembanding:  2 Ayat:  231 Ayat Pembanding:  267 LCS:  wa Ealima w^A@ >an~ {ll~ah
Surat:  2 Surat Pembanding:  2 Ayat:  231 Ayat Pembanding:  272 LCS:  kumo wa maA
Surat:  2 Surat Pembanding:  2 Ayat:  231 Ayat Pembanding:  278 LCS:  {t~aqaY` wA@ {ll~ah wa
Surat:  2 Surat Pembanding:  2 Ayat:  231 Ayat Pembanding:  281 LCS:  wa {t~aqaY` wA@
Surat:  2 Surat Pembanding:  2 Ayat:  231 Ayat Pembanding:  282 LCS:  wa {t~aqaY` wA@ {ll~ah wa
Surat:  2 Surat Pembanding:  2 Ayat:  231 Ayat Pembanding:  283 LCS:  hu, wa laA
Surat:  2 Surat Pembanding:  2 Ayat:  231 Ayat Pembanding:  286 LCS:  bi hi. wa
Surat:  2 Surat Pembanding:  3 Ayat:  231 Ayat Pembanding:  7 LCS:  {lo kita`b wa
Surat:  2 Surat Pembanding:  2 Ayat:  232 Ayat Pembanding:  233 LCS:  bi {lo m~aEoruwf
Surat:  2 Surat Pembanding:  2 Ayat:  232 Ayat Pembanding:  234 LCS:  balaga na >ajal hun~a fa laA
Surat:  2 Surat Pembanding:  2 A

Surat:  2 Surat Pembanding:  2 Ayat:  249 Ayat Pembanding:  250 LCS:  jaAluwt wa jund hi. qaAla
Surat:  2 Surat Pembanding:  2 Ayat:  249 Ayat Pembanding:  251 LCS:  bi <i*on {ll~ah wa
Surat:  2 Surat Pembanding:  2 Ayat:  249 Ayat Pembanding:  257 LCS:  {l~a*iY 'aAmana wA@
Surat:  2 Surat Pembanding:  2 Ayat:  249 Ayat Pembanding:  264 LCS:  {l~a*iY 'aAmana wA@
Surat:  2 Surat Pembanding:  2 Ayat:  249 Ayat Pembanding:  277 LCS:  {l~a*iY 'aAmana wA@
Surat:  2 Surat Pembanding:  2 Ayat:  249 Ayat Pembanding:  278 LCS:  {l~a*iY 'aAmana wA@
Surat:  2 Surat Pembanding:  2 Ayat:  249 Ayat Pembanding:  282 LCS:  {ll~ah wa {ll~ah
Surat:  2 Surat Pembanding:  2 Ayat:  249 Ayat Pembanding:  286 LCS:  laA TaAqap la naA
Surat:  2 Surat Pembanding:  2 Ayat:  250 Ayat Pembanding:  264 LCS:  {lo qawom {lo ka`firuwn
Surat:  2 Surat Pembanding:  2 Ayat:  250 Ayat Pembanding:  286 LCS:  naSara naA EalaY` {lo qawom {lo ka`firuwn
Surat:  2 Surat Pembanding:  2 Ayat:  251 Ayat Pembanding:  253 LCS:  wa l

Surat:  2 Surat Pembanding:  2 Ayat:  272 Ayat Pembanding:  279 LCS:  laA Zalama wna
Surat:  2 Surat Pembanding:  2 Ayat:  272 Ayat Pembanding:  281 LCS:  laA Zalama wna
Surat:  2 Surat Pembanding:  2 Ayat:  272 Ayat Pembanding:  284 LCS:  man $aA^'a wa
Surat:  2 Surat Pembanding:  2 Ayat:  273 Ayat Pembanding:  284 LCS:  fiY {lo >aroD
Surat:  2 Surat Pembanding:  3 Ayat:  273 Ayat Pembanding:  5 LCS:  fiY {lo >aroD
Surat:  2 Surat Pembanding:  2 Ayat:  274 Ayat Pembanding:  277 LCS:  la humo >ajor humo Eind rab~ himo wa laA xawof EalaY` himo wa laA humo yaHozun wna
Surat:  2 Surat Pembanding:  2 Ayat:  275 Ayat Pembanding:  276 LCS:  {l r~ibaw`A wa
Surat:  2 Surat Pembanding:  2 Ayat:  275 Ayat Pembanding:  285 LCS:  min rab~ hi.
Surat:  2 Surat Pembanding:  2 Ayat:  277 Ayat Pembanding:  278 LCS:  {l~a*iY 'aAmana wA@
Surat:  2 Surat Pembanding:  2 Ayat:  278 Ayat Pembanding:  282 LCS:  ya`^ >ay~uhaA {l~a*iY 'aAmana
Surat:  2 Surat Pembanding:  2 Ayat:  279 Ayat Pembanding:  281 LCS: 

In [29]:
import pandas as pd
data_frame = pd.DataFrame(list_hasil, columns=['surat','surat_pembanding', 'ayat', 'ayat_pembanding', 'jumlah_kata', 'lcs'])
data_frame

,surat,surat_pembanding,ayat,ayat_pembanding,jumlah_kata,lcs
0,1,1,1,3,4,{l r~aHoma`n {l r~aHiym
1,1,2,1,163,4,{l r~aHoma`n {l r~aHiym
2,1,2,2,131,3,rab~ {lo Ea`lamiyn
3,1,2,3,163,4,{l r~aHoma`n {l r~aHiym
4,1,2,7,38,4,EalaY` himo wa laA
5,1,2,7,62,4,EalaY` himo wa laA
6,1,2,7,112,4,EalaY` himo wa laA
7,1,2,7,120,3,wa laA {l
8,1,2,7,160,3,EalaY` himo wa
9,1,2,7,167,3,EalaY` himo wa


In [30]:
# from pandas import ExcelWriter

# writer = ExcelWriter('data_test_arab_lcs.xlsx')
# data_frame.to_excel(writer,'arab')
# writer.save()

In [31]:
data_frame.sort_values(by=['lcs', 'surat', 'ayat'])

,surat,surat_pembanding,ayat,ayat_pembanding,jumlah_kata,lcs
220,2,2,20,70,3,$aA^'a {ll~ah la
801,2,2,70,220,3,$aA^'a {ll~ah la
587,2,2,48,170,3,$aYo' wa laA
1370,2,2,123,170,3,$aYo' wa laA
700,2,2,60,249,3,$ariba wA@ min
752,2,2,62,256,3,'aAmana bi {ll~ah
1792,2,2,177,256,3,'aAmana bi {ll~ah
2261,2,2,232,256,3,'aAmana bi {ll~ah
2370,2,2,256,264,3,'aAmana bi {ll~ah
2371,2,2,256,285,3,'aAmana bi {ll~ah


In [32]:
data_frame_distinct = data_frame.drop_duplicates(subset='lcs', keep='first', inplace=False)
data_frame_distinct

,surat,surat_pembanding,ayat,ayat_pembanding,jumlah_kata,lcs
0,1,1,1,3,4,{l r~aHoma`n {l r~aHiym
2,1,2,2,131,3,rab~ {lo Ea`lamiyn
4,1,2,7,38,4,EalaY` himo wa laA
7,1,2,7,120,3,wa laA {l
8,1,2,7,160,3,EalaY` himo wa
13,2,2,2,66,3,l~i lo mut~aqiyn
14,2,2,3,4,4,{l~a*iY 'aAmana wna bi
15,2,2,3,43,3,{l Salaw`p wa
17,2,2,3,57,3,maA razaqa na`
19,2,2,3,85,3,'aAmana wna bi


In [33]:
data_frame_distinct.drop(['surat', 'surat_pembanding', 'ayat', 'ayat_pembanding'], axis=1, inplace=True)

C:\Users\DEVI\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [34]:
def join_verses_for_lcs(lcs: str):
    matches = data_frame.loc[data_frame['lcs'] == lcs]
    verses1 = matches.apply(lambda row: '%s:%s' % (row['surat'], row['ayat']), axis=1)
    verses2 = matches.apply(lambda row: '%s:%s' % (row['surat_pembanding'], row['ayat_pembanding']), axis=1)
    verseset = list(verses1) + list(verses2) # concatenate lists
    verseset = list(dict.fromkeys(verseset)) # remove duplicate verses
    return verseset

# join_verses_for_lcs('yang kami')
data_frame_distinct = data_frame_distinct.copy()
data_frame_distinct['first_verse'] = data_frame_distinct.apply(lambda row: join_verses_for_lcs(row['lcs'])[0], axis=1)
data_frame_distinct['verses'] = data_frame_distinct.apply(lambda row: ', '.join(join_verses_for_lcs(row['lcs'])), axis=1)
data_frame_distinct['verse_count'] = data_frame_distinct.apply(lambda row: len(join_verses_for_lcs(row['lcs'])), axis=1)
data_frame_distinct

,jumlah_kata,lcs,first_verse,verses,verse_count
0,4,{l r~aHoma`n {l r~aHiym,1:1,"1:1, 1:3, 2:163",3
2,3,rab~ {lo Ea`lamiyn,1:2,"1:2, 2:131",2
4,4,EalaY` himo wa laA,1:7,"1:7, 2:38, 2:62, 2:112, 2:262, 2:274, 2:277",7
7,3,wa laA {l,1:7,"1:7, 2:120",2
8,3,EalaY` himo wa,1:7,"1:7, 2:38, 2:62, 2:112, 2:160, 2:167, 2:262, 2...",9
13,3,l~i lo mut~aqiyn,2:2,"2:2, 2:66",2
14,4,{l~a*iY 'aAmana wna bi,2:3,"2:3, 2:4",2
15,3,{l Salaw`p wa,2:3,"2:3, 2:43, 2:45, 2:83, 2:110, 2:177, 2:238, 2:277",8
17,3,maA razaqa na`,2:3,"2:3, 2:57, 2:172, 2:254",4
19,3,'aAmana wna bi,2:3,"2:3, 2:4, 2:85, 2:121",4


In [35]:
from pandas import ExcelWriter

writer = ExcelWriter('data_300_arab_lem.xlsx')
data_frame_distinct.to_excel(writer,'arab')
writer.save()

In [36]:
buck2uni = {"'": u"\u0621", # hamza-on-the-line
            "|": u"\u0622", # madda
            ">": u"\u0623", # hamza-on-'alif
            "&": u"\u0624", # hamza-on-waaw
            "<": u"\u0625", # hamza-under-'alif
            "}": u"\u0626", # hamza-on-yaa'
            "A": u"\u0627", # bare 'alif
            "b": u"\u0628", # baa'
            "p": u"\u0629", # taa' marbuuTa
            "t": u"\u062A", # taa'
            "v": u"\u062B", # thaa'
            "j": u"\u062C", # jiim
            "H": u"\u062D", # Haa'
            "x": u"\u062E", # khaa'
            "d": u"\u062F", # daal
            "*": u"\u0630", # dhaal
            "r": u"\u0631", # raa'
            "z": u"\u0632", # zaay
            "s": u"\u0633", # siin
            "$": u"\u0634", # shiin
            "S": u"\u0635", # Saad
            "D": u"\u0636", # Daad
            "T": u"\u0637", # Taa'
            "Z": u"\u0638", # Zaa' (DHaa')
            "E": u"\u0639", # cayn
            "g": u"\u063A", # ghayn
            "_": u"\u0640", # taTwiil
            "f": u"\u0641", # faa'
            "q": u"\u0642", # qaaf
            "k": u"\u0643", # kaaf
            "l": u"\u0644", # laam
            "m": u"\u0645", # miim
            "n": u"\u0646", # nuun
            "h": u"\u0647", # haa'
            "w": u"\u0648", # waaw
            "Y": u"\u0649", # 'alif maqSuura
            "y": u"\u064A", # yaa'
            "F": u"\u064B", # fatHatayn
            "N": u"\u064C", # Dammatayn
            "K": u"\u064D", # kasratayn
            "a": u"\u064E", # fatHa
            "u": u"\u064F", # Damma
            "i": u"\u0650", # kasra
            "~": u"\u0651", # shaddah
            "o": u"\u0652", # sukuun
            "`": u"\u0670", # dagger 'alif
            "{": u"\u0671", # waSla
}

In [37]:
def transString(string, reverse=0):
    '''Given a Unicode string, transliterate into Buckwalter. To go from
    Buckwalter back to Unicode, set reverse=1'''

    for k, v in buck2uni.items():
      if not reverse:
            string = string.replace(v, k)
      else:
            string = string.replace(k, v)

    return string

In [46]:
print(transString("{l~a*iY 'aAmana wA@ wa Eamila wA@ {l S~a`liHa`t", 1))

ٱلَّذِى ءَامَنَ وا@ وَ عَمِلَ وا@ ٱل صَّٰلِحَٰت
